In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

# import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

**Import Essential Libraries**

In [ ]:
# Fundamental Libraries
import numpy as np
import pandas as pd

# Ordinal Encoding and train test split
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import train_test_split

# XGB Regressor
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error

**Load data**

In [ ]:
train = pd.read_csv("/kaggle/input/30daysofmlraw/train.csv", index_col=0)
test = pd.read_csv("/kaggle/input/30daysofmlraw/test.csv", index_col=0)

# train.head()

**Separating target from the feature**

In [ ]:
y = train['target']
X_train_full = train.drop(['target'], axis=1)

# print(y_train)
# print(X_train)

**Handle categorical variables**

In [ ]:
object_col = [col for col in X_train_full.columns if 'cat' in col]

# Ordinal Encoding the categorical variable since these are orderly related
X = X_train_full.copy()
X_test = test.copy()
oe = OrdinalEncoder()
X[object_col] = oe.fit_transform(X_train_full[object_col])
X_test[object_col] = oe.transform(test[object_col])

X.head()

**Splitting dataset**

In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(X,y, random_state=0)

**Training a Model**

In [ ]:
# XGBRegressor
model = XGBRegressor(n_estimators=1000, learning_rate=0.1, n_jobs=4)

model.fit(X_train, y_train, early_stopping_rounds=5, eval_set=[(X_valid, y_valid)], verbose=False)
preds = model.predict(X_valid)
print(mean_squared_error(preds, y_valid, squared=False))

**Generating predictions**


In [ ]:
preds_2 = model.predict(X_test)

# Save the predictions to CSV file
output = pd.DataFrame({'Id': X_test.index,'target': preds_2})
output.to_csv('submission.csv', index=False)